In [1]:
import os
import faiss
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

## Этап 1. Загрузка текстов из файлов

In [2]:
TEXT_FILES_DIR = "garant"

In [3]:
def load_texts_from_folder(folder_path):
    texts = []
    filenames = []
    for file in tqdm(sorted(os.listdir(folder_path)), desc="Загрузка текстов"):
        if file.endswith(".txt"):
            file_path = os.path.join(folder_path, file)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read().strip()
                texts.append(text)
                filenames.append(file)
    return texts, filenames

In [4]:
texts, filenames = load_texts_from_folder(TEXT_FILES_DIR)

Загрузка текстов: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1628/1628 [00:17<00:00, 93.70it/s]


## Этап 2. Создание эмбеддингов

In [5]:
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(MODEL_NAME)

In [6]:
text_embeddings = model.encode(texts, batch_size=16, convert_to_numpy=True, show_progress_bar=True)
print(f"\nРазмерность эмбеддингов: {text_embeddings.shape}")

Batches:   0%|          | 0/102 [00:00<?, ?it/s]


Размерность эмбеддингов: (1628, 384)


## Этап 3: Создание FAISS-индекса

FAISS используется для быстрого поиска среди эмбеддингов

In [7]:
dimension = text_embeddings.shape[1]

Создаём FAISS-индекс

In [8]:
index = faiss.IndexFlatL2(dimension)  # поиск с использованием L2-дистанции (евклидово расстояние)
index.add(text_embeddings)

Сохраняем индекс для последующего использования

In [9]:
faiss.write_index(index, "results_BERT/text_index.faiss")
np.save("results_BERT/filenames.npy", np.array(filenames))  # сохраняем список имён файлов

RuntimeError: Error in faiss::FileIOWriter::FileIOWriter(const char*) at /project/faiss/faiss/impl/io.cpp:101: Error: 'f' failed: could not open results_BERT/text_index.faiss for writing: No such file or directory

## Этап 4: Функция поиска похожих текстов

In [ ]:
def find_similar_texts(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True) # преобразуем запрос в эмбеддинг
    distances, indices = index.search(query_embedding, top_k) # выполняем поиск в FAISS

    print("Наиболее похожие тексты:")
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. {filenames[idx]} (дистанция: {distances[0][i]:.4f})")
        print(texts[idx][:300] + "...")  # выводим первые 300 символов текста
        print("-" * 80)

In [ ]:
query_text = "Международный день инвалидов в Хабаровске"
find_similar_texts(query_text, top_k=20)